In [1]:
# Imports
from subsbml import System, createSubsystem, combineSystems, createNewSubsystem, createBasicSubsystem, SimpleModel, SimpleReaction
import numpy as np
import pylab as plt

from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import libsbml
import bokeh.io
import bokeh.plotting

## Cells B and P communicating with each other ##
Cell_B = Bistable SWitch

Cell_P = Produce2N

External environment = rR1 reservoir

In [2]:
ss1 = createSubsystem ("SBML_Models/BiSwitch_CRN.xml")
ss2 = createSubsystem ("SBML_Models/external_environment.xml")

# Membrane 
mb1 = createSubsystem("SBML_Models/rR1_membrane1_detailed.xml", membrane = True)
### Cell B ###
cell_B = System("cell_B")

cell_B.setInternal([ss1])
cell_B.setExternal([ss2])
cell_B.setMembrane(mb1)
cell_B_model = cell_B.getModel()

cell_B_model.renameCompartments('cell_B_external', 'actual_external')

##### Set Species Amount #####
cell_B_model.setSpeciesAmount('rna_rR1_e', 0, compartment = 'actual_external')
#cell_B_model.setSpeciesAmount('rna_rR1', 100, compartment = 'cell_P_external')

cell_B_model.setSpeciesAmount('Core1_OFF', 6e3, compartment = 'cell_B_internal')
cell_B_model.setSpeciesAmount('Core2_OFF', 4e3, compartment = 'cell_B_internal')
cell_B_model.setSpeciesAmount('dna_dA2', 6e3, compartment = 'cell_B_internal')
cell_B_model.setSpeciesAmount('dna_dA1', 6e3, compartment = 'cell_B_internal')
cell_B_model.setSpeciesAmount('protein_RNAseH', 40, compartment = 'cell_B_internal')
cell_B_model.setSpeciesAmount('protein_RNAP', 300, compartment = 'cell_B_internal')

cell_B_model.writeSBML('SBML_Models/cell_B_model.xml')
print (cell_B)

The subsystem from SBML_Models/rR1_membrane1_detailed.xml has multiple compartments


In [3]:
ss3 = createSubsystem("SBML_Models/Full_Produce.xml")

# Membrane
mb2 = createSubsystem("SBML_Models/rR1_P_membrane1_detailed.xml", membrane = True)
mb3 = createSubsystem("SBML_Models/rR1_Broc_membrane1_detailed.xml", membrane = True)

### Cell P ###
cell_P =  System('cell_P',ListOfInternalSubsystems = [ss3],
                 ListOfExternalSubsystems = [ss2],
                 ListOfMembraneSubsystems = [mb2,mb3])


cell_P_model = cell_P.getModel()

cell_P_model.renameCompartments('cell_P_external', 'actual_external')

##### Set Species Amount #####
cell_P_model.setSpeciesAmount('rna_rR1_e', 0, compartment = 'actual_external')
#cell_P_model.setSpeciesAmount('rna_rR1', 100, compartment = 'cell_P_external')

cell_P_model.setSpeciesAmount('Prod2_OFF', 4e3, compartment = 'cell_P_internal')
cell_P_model.setSpeciesAmount('dna_dAP', 4e3, compartment = 'cell_P_internal')
cell_P_model.setSpeciesAmount('protein_RNAseH', 40, compartment = 'cell_P_internal')
cell_P_model.setSpeciesAmount('protein_RNAP', 300, compartment = 'cell_P_internal')
cell_P_model.setSpeciesAmount('DFHBI', 6e4, compartment = 'cell_P_internal')

cell_P_model.writeSBML('SBML_Models/cell_P_model.xml')

The subsystem from SBML_Models/rR1_P_membrane1_detailed.xml has multiple compartments
The subsystem from SBML_Models/rR1_Broc_membrane1_detailed.xml has multiple compartments


1

In [4]:
combined_cellsBP = createNewSubsystem()
combined_cellsBP.combineSubsystems([cell_B_model, cell_P_model], verbose = True)
#combined_cellsBP.writeSBML('SBML_Models/combined_BP.xml')
#combined_cellsBP.changeLog

The Ids are identical, renaming stopped.


C:\Users\Jeremiah\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:646: UserWarning: Not all of the Subsystems being combined are in the same Compartment
C:\Users\Jeremiah\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:987: UserWarning: Annotated propensity updated to: <PropensityType> type=massaction k=0.00394  </PropensityType>
C:\Users\Jeremiah\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:987: UserWarning: Annotated propensity updated to: <PropensityType> type=massaction k=0.0696  </PropensityType>
C:\Users\Jeremiah\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:987: UserWarning: Annotated propensity updated to: <PropensityType> type=massaction k=38.857142857142854  </PropensityType>
C:\Users\Jeremiah\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:987: UserWarning: Annotated propensity updated to: <PropensityType> type=massaction k=10.0  </PropensityType>
C:\Users\

<SBMLDocument>

In [5]:
# Cell_B Identifiers #

B_id1 = combined_cellsBP.getSpeciesByName('complex_Core1_ON').getId()
B_id2 = combined_cellsBP.getSpeciesByName('Core1_OFF').getId()
B_id3 = combined_cellsBP.getSpeciesByName('complex_Core2_ON').getId()
B_id4 = combined_cellsBP.getSpeciesByName('Core2_OFF').getId()
B_id5 = combined_cellsBP.getSpeciesByName("rna_rR2").getId()
B_id6 = combined_cellsBP.getSpeciesByName("rna_rR1", compartment = "cell_B_internal").getId()


B_id8 = combined_cellsBP.getSpeciesByName("complex_Core2_AI").getId()
B_id9 = combined_cellsBP.getSpeciesByName("complex_Core1_AI").getId()
B_id10 = combined_cellsBP.getSpeciesByName("dna_dA2").getId()
B_id11 = combined_cellsBP.getSpeciesByName("dna_dA1").getId()

In [6]:
# Cell_P Identifiers #

P_id1 = combined_cellsBP.getSpeciesByName('complex_Prod2_ON').getId()
P_id2 = combined_cellsBP.getSpeciesByName('Prod2_OFF').getId()
P_id3 = combined_cellsBP.getSpeciesByName('DFHBI').getId()
P_id4 = combined_cellsBP.getSpeciesByName('BrocApt').getId()
P_id5 = combined_cellsBP.getSpeciesByName("complex_BrocApt_DFHBI").getId()
P_id6 = combined_cellsBP.getSpeciesByName("rna_rR1P").getId()
P_id7 = combined_cellsBP.getSpeciesByName("BrocApt_e", compartment="actual_external").getId()

BP_id1 = combined_cellsBP.getSpeciesByName("rna_rR1_e").getId()

print (BP_id1)

rna_rR1_e_external_environment_model_1_combined_1_combined


In [7]:
#### Set Species Amount for Combined cells ######
#combined_cellsBP.setSpeciesAmount('rna_rR1_e', 250)
combined_cellsBP.setSpeciesAmount('rna_rR1_e', 0)


#### Cell B #####
combined_cellsBP.setSpeciesAmount('Core1_OFF', 1.2, compartment = 'cell_B_internal')
combined_cellsBP.setSpeciesAmount('Core2_OFF', 0.99, compartment = 'cell_B_internal')
combined_cellsBP.setSpeciesAmount('dna_dA2', 1.2, compartment = 'cell_B_internal')
combined_cellsBP.setSpeciesAmount('dna_dA1', 1.2, compartment = 'cell_B_internal')
combined_cellsBP.setSpeciesAmount('protein_RNAseH', 0.0044, compartment = 'cell_B_internal')
combined_cellsBP.setSpeciesAmount('protein_RNAP', 0.030, compartment = 'cell_B_internal')

#### Cell P ####
combined_cellsBP.setSpeciesAmount('Prod2_OFF', 0.035, compartment = 'cell_P_internal')
combined_cellsBP.setSpeciesAmount('dna_dAP', 0.035, compartment = 'cell_P_internal')
combined_cellsBP.setSpeciesAmount('protein_RNAseH', 0.0044, compartment = 'cell_P_internal')
combined_cellsBP.setSpeciesAmount('protein_RNAP', 0.030, compartment = 'cell_P_internal')
combined_cellsBP.setSpeciesAmount('DFHBI', 1, compartment = 'cell_P_internal')

combined_cellsBP.writeSBML('SBML_Models/combined_BP.xml')
combined_cellsBP.changeLog

{'actual_external': 'actual_external_combined',
 'BrocApt_e_external_environment_model': 'BrocApt_e_external_environment_model_1_combined',
 'rna_rR1_e_external_environment_model_1_combined': 'rna_rR1_e_external_environment_model_1_combined_1_combined'}

In [8]:
# Simulate with Bioscrape #
timepoints = np.linspace(0,28800,1000)
B,_ = combined_cellsBP.simulateWithBioscrape(timepoints)
timepoints = timepoints/3600

# For label convenience
x = 'Time (hours)'
y = 'Concentration (uM)'

bokeh.io.output_notebook()
a = bokeh.plotting.figure(title="Bistable Switch",plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
a.circle(timepoints, B[B_id1], legend_label = "Core1 ON" , color = "blue")
a.circle(timepoints, B[B_id3], legend_label = "Core2 ON", color = "red")
a.legend.click_policy="hide"
a.legend.location="bottom_right"

b = bokeh.plotting.figure(title="Produce2N",plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
b.circle(timepoints, B[P_id1], legend_label = "Prod2 ON" , color = "blue")
b.circle(timepoints, B[P_id2], legend_label = "Prod2 OFF", color = "red")
b.legend.click_policy="hide"
b.legend.location="bottom_right"

bokeh.io.show(row(a, b))
warnings.filterwarnings("ignore")

C:\Users\Jeremiah\anaconda3\lib\site-packages\bioscrape\sbmlutil.py:93: UserWarning: SBML model contains reversible reaction!
Please check rate expressions and ensure they are non-negative before doing stochastic simulations.
  'Please check rate expressions and ensure they are non-negative before doing '+
C:\Users\Jeremiah\anaconda3\lib\site-packages\bioscrape\sbmlutil.py:208: UserWarning: Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. Refer to the bioscrape wiki for more information.
  warnings.warn('Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. ' +


Loading BokehJS ...

In [9]:
a = bokeh.plotting.figure(title="BiSwitch rR1", plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
a.circle(timepoints, B[B_id6], legend_label = "rR1 internal B" , color = "blue")
a.legend.click_policy="hide"
a.legend.location="top_right"

b = bokeh.plotting.figure(title="Prod2 rR1",plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
b.circle(timepoints, B[P_id6], legend_label = "rR1 internal P" , color = "darkgreen")
b.legend.click_policy="hide"
b.legend.location="top_right"

c = bokeh.plotting.figure(title="External rR1",plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
c.circle(timepoints, B[BP_id1], legend_label = "rR1 external", color = "orange")
c.legend.click_policy="hide"
c.legend.location="bottom_right"

d = bokeh.plotting.figure(title="External Broccoli",plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
d.circle(timepoints, B[P_id7], legend_label = "BrocApt external", color = "red")
#d.circle(timepoints, B[P_id4], legend_label = "BrocApt", color = "green")
d.legend.click_policy="hide"
d.legend.location="bottom_right"

bokeh.io.show(row(a, b, c, d))
warnings.filterwarnings("ignore")

In [10]:
P_id9 = combined_cellsBP.getSpeciesByName("complex_Prod2_AI").getId()

e = bokeh.plotting.figure(title="Produce2N A:I",plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
e.circle(timepoints, B[P_id9], legend_label = "dA1:rR1", color = "red")
e.legend.click_policy="hide"
e.legend.location="bottom_right"

f = bokeh.plotting.figure(title="Produce2N fluorescence",plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
f.circle(timepoints, B[P_id5], legend_label = "Fluorescence", color = "lightgreen")
f.legend.click_policy="hide"
f.legend.location="bottom_right"

bokeh.io.show(row(e, f))
warnings.filterwarnings("ignore")